In [1]:
import csv
import numpy as np
import pandas as pd
import torch
import torch.nn as nn

# featurizer class definition

class Featurizer():
    
    def __call__(self, df):
        fingerprints = []
        labels = []
        
        df = df[df['Ki']<=1e4]
        df = df[df['Ki']>0.01]
        labels = df['Ki']
        
        fp = []
        for index, row in df.iterrows():
            fp = row[1:]
            fingerprints.append(fp)
            
        fingerprints = np.array(fingerprints)
        labels = np.array(labels)
        return fingerprints, labels

In [2]:
# text file to DataFrame object

filename = '../cleaned_datasets/d2_Sub_clean.csv'
df = pd.read_csv(filename)
df = df.dropna()

featurizer = Featurizer()
fp_train, ki_train = featurizer(df)

fp_train = torch.from_numpy(fp_train)
ki_train = torch.from_numpy(ki_train)

assert fp_train.shape[0] == ki_train.shape[0], 'X_train and y_train rows do not match'

In [3]:
fp_train.shape

torch.Size([9249, 139])

In [4]:
# data loader

from torch.utils.data import DataLoader, TensorDataset

tensor_ds = TensorDataset(fp_train, ki_train)
train = tensor_ds

train_dataloader = DataLoader(train, batch_size=64, shuffle=True)

In [5]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cpu


In [6]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable

# define variational autoencoder class

class Encoder(nn.Module):
    def __init__(self, input_size, output_size):
        super(Encoder, self).__init__()
        self.fc1 = nn.Linear(input_size, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 16)
        self.fc41 = nn.Linear(16, output_size)
        self.fc42 = nn.Linear(16, output_size)
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        h1 = self.relu(self.fc1(x))
        h2 = self.relu(self.fc2(h1))
        h3 = self.relu(self.fc3(h2))
        mu = self.fc41(h3)
        logvar = self.fc41(h3)
        return mu, logvar

class Decoder(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(Decoder, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, 64)
        self.fc3 = nn.Linear(64, 128)
        self.fc4 = nn.Linear(128, output_size)
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        h1 = self.relu(self.fc1(x))
        h2 = self.relu(self.fc2(h1))
        h3 = self.relu(self.fc3(h2))
        out = self.fc4(h3)
        return self.sigmoid(out)

class VAE(nn.Module):
    def __init__(self, input_size, latent_size):
        torch.set_default_dtype(torch.float64)
        super(VAE, self).__init__()
        self.encoder = Encoder(input_size, latent_size)
        self.decoder = Decoder(latent_size, 128, input_size)

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        return eps.mul(std).add_(mu)

    def forward(self, x):
        mu, logvar = self.encoder(x)
        z = self.reparameterize(mu, logvar)
        return self.decoder(z), mu, logvar


In [7]:
# define VAE loss function

class VAELoss(nn.Module):
    def __init__(self):
        super(VAELoss, self).__init__()

    def forward(self, recon_x, x, mu, logvar):
        BCE = nn.functional.binary_cross_entropy(recon_x, x, reduction='sum')
        KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
        return BCE + KLD

In [8]:
# train

def train_VAE(train_dataloader, fp_len, epochs=40, device=device, code_len=8):
    model = VAE(fp_len, code_len)
    criterion = VAELoss()
    if device == 'cuda':
        model.cuda()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.00001)
    
    for epoch in range(epochs):
        for (fp, _) in train_dataloader:
            if device == 'cuda':
                fp = fp.cuda()
            encoded, mu, logvar = model(fp)
            loss = criterion(encoded, fp, mu, logvar)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        print(f'Epoch: {epoch+1}, Loss: {loss.item():.4f}')
    return model

In [9]:
model = train_VAE(train_dataloader, fp_len=fp_train.size(dim=1))

Epoch: 1, Loss: 3159.2476
Epoch: 2, Loss: 3114.5109
Epoch: 3, Loss: 3035.9676
Epoch: 4, Loss: 2901.1650
Epoch: 5, Loss: 2629.7011
Epoch: 6, Loss: 2263.7502
Epoch: 7, Loss: 1878.7771
Epoch: 8, Loss: 1478.6566
Epoch: 9, Loss: 1232.4461
Epoch: 10, Loss: 1022.3400
Epoch: 11, Loss: 974.5571
Epoch: 12, Loss: 829.9723
Epoch: 13, Loss: 779.2710
Epoch: 14, Loss: 844.5443
Epoch: 15, Loss: 685.2414
Epoch: 16, Loss: 745.9238
Epoch: 17, Loss: 762.9936
Epoch: 18, Loss: 721.9691
Epoch: 19, Loss: 706.0238
Epoch: 20, Loss: 637.0776
Epoch: 21, Loss: 667.4802
Epoch: 22, Loss: 594.7723
Epoch: 23, Loss: 706.7089
Epoch: 24, Loss: 622.7901
Epoch: 25, Loss: 660.6580
Epoch: 26, Loss: 670.1354
Epoch: 27, Loss: 652.3331
Epoch: 28, Loss: 663.0715
Epoch: 29, Loss: 627.8684
Epoch: 30, Loss: 638.0880
Epoch: 31, Loss: 754.6127
Epoch: 32, Loss: 682.6509
Epoch: 33, Loss: 668.4012
Epoch: 34, Loss: 662.4492
Epoch: 35, Loss: 582.8796
Epoch: 36, Loss: 636.8236
Epoch: 37, Loss: 641.5531
Epoch: 38, Loss: 638.8915
Epoch: 39, 

In [10]:
# encode all data to latent space
encoded = []
for fp in fp_train:
    encoder = model.encoder
    if device == 'cuda':
        fp = fp.cuda()
        mu, logvar = encoder(fp)
        latent = model.reparameterize(mu, logvar)
        encoded.append(latent.detach().cpu().numpy())
    else:
        mu, logvar = encoder(fp)
        latent = model.reparameterize(mu, logvar)
        encoded.append(latent.detach().numpy())

In [11]:
encoded = np.array(encoded)

In [12]:
# filter for active compounds only

activity = (ki_train < 10)
activity = np.array(activity)
vae_results = pd.DataFrame(encoded)
vae_results['activity'] = activity

print(encoded.shape, activity.shape)

(9249, 8) (9249,)


In [36]:
# latent space data prep

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(encoded, activity, test_size=0.3, random_state=42)
                                    
X_train = torch.from_numpy(X_train)
X_test = torch.from_numpy(X_test)
y_train = torch.from_numpy(y_train)
y_test = torch.from_numpy(y_test)

assert X_train.shape[0] == y_train.shape[0], 'X_train and y_train rows do not match'

train_dataset = TensorDataset(X_train, y_train)
                                    
latent_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True)

In [14]:
# define prediction neural net class

class Predictor(nn.Module):
    def __init__(self, hidden_size=32, input_size=8):
        super(Predictor, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, 2)
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()
        
    def forward(self, x):
        h1 = self.relu(self.fc1(x))
        h2 = self.relu(self.fc2(h1))
        out = self.sigmoid(h2)
        return out

In [15]:
def train_predictor(latent_dataloader, epochs=100, device=device):
    
    # TODO

In [16]:
# train predictor nn

predictor = train_predictor(latent_dataloader)


ValueError: Using a target size (torch.Size([64, 8])) that is different to the input size (torch.Size([64, 2])) is deprecated. Please ensure they have the same size.

In [ ]:
# plot latent space

from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt

# pca reduction

pca = PCA(n_components=2)
pca_results = pca.fit_transform(encoded)
pca_results = pd.DataFrame(pca_results)
pca_results['activity'] = activity


In [ ]:
# t-SNE reduction

print('...tSNE...')
tsne = tsne = TSNE(perplexity=20, learning_rate=100, verbose=1)
tsne_results = pd.DataFrame(tsne.fit_transform(encoded))
tsne_results['activity'] = activity

In [ ]:
import seaborn as sns

# plot PCA

sns.set_style('whitegrid')
colors = ['#D3D3D3', '#880808']
sns.scatterplot(x=0, y=1, hue='activity', data=pca_results, marker='.', palette=colors)
plt.title('VAE PCA')
plt.xlabel('PC1')
plt.ylabel('PC2')

In [ ]:
# plot t-SNE

sns.scatterplot(x=0, y=1, hue='activity', data=tsne_results, marker='.', palette=colors)
plt.title('VAE t-SNE')
plt.xlabel('Ax1')
plt.ylabel('Ax2')